$$
\begin{cases}
\dot{x}_i = v \cos \theta_i i \\
\dot{y}_i = v \sin \theta_i i \\
\dot{\theta}_i = \omega_i + K_1 \sum_{j=1}^N A_{ij} \sin \left( \theta_j - \theta_i \right) + K_2 \sum_{j=1}^N \sum_{k=1}^N G_{ijk} \sin \left( \theta_j + \theta_k - 2\theta_i \right)
\end{cases}
\quad
A_{ij} = H\left( d_1 - d_{ij}(t) \right) =
\begin{cases}
1, & d_{ij}(t) \leq d_1 \\
0, & \text{else}
\end{cases}
\quad
G_{ijk} = H\left( d_2 - d_{ij}(t) \right) H\left( d_2 - d_{jk}(t) \right) H\left( d_2 - d_{ik}(t) \right)
\quad
d_{ij} = \sqrt{\left[ x_i(t) - x_j(t) \right]^2 + \left[ y_i(t) - y_j(t) \right]^2}
$$

In [1]:
from main import *

In [2]:
model = ThreeBody(strengthLambda1=0.09, strengthLambda2=0.09, 
                  distanceD1=0.5, distanceD2=0.5, agentsNum=200, boundaryLength=5, gamma=0.52,
                  tqdm=True, savePath="./data", overWrite=True)
# model.tempForK = model.tempForK

In [3]:
model.run(50000)

  0%|          | 0/50000 [00:00<?, ?it/s]

In [4]:
import pandas as pd
from matplotlib.animation import FuncAnimation

def draw_mp4(model):

    targetPath = f"./data/{model}.h5"
    totalPositionX = pd.read_hdf(targetPath, key="positionX")
    totalPhaseTheta = pd.read_hdf(targetPath, key="phaseTheta")
    totalPointTheta = pd.read_hdf(targetPath, key="pointTheta")
    TNum = totalPositionX.shape[0] // model.agentsNum
    totalPositionX = totalPositionX.values.reshape(TNum, model.agentsNum, 2)
    totalPhaseTheta = totalPhaseTheta.values.reshape(TNum, model.agentsNum)
    totalPointTheta = totalPointTheta.values.reshape(TNum, model.agentsNum)
    shift = 0
    class1, class2 = (
        np.concatenate([np.ones(model.agentsNum // 2), np.zeros(model.agentsNum // 2)]).astype(bool), 
        np.concatenate([np.zeros(model.agentsNum // 2), np.ones(model.agentsNum // 2)]).astype(bool)
    )

    def plot_frame(i):
        pbar.update(1)
        positionX = totalPositionX[i]
        phaseTheta = totalPhaseTheta[i]
        fig.clear()
        ax1 = plt.subplot(1, 2, 1)
        ax1.quiver(
            positionX[class1, 0], positionX[class1, 1],
            np.cos(phaseTheta[class1]), np.sin(phaseTheta[class1]), color='tomato'
        )
        ax1.quiver(
            positionX[class2, 0], positionX[class2, 1],
            np.cos(phaseTheta[class2]), np.sin(phaseTheta[class2]), color='dodgerblue'
        )
        limShift = 0
        ax1.set_xlim(0 - limShift, model.boundaryLength + limShift)
        ax1.set_ylim(0 - limShift, model.boundaryLength + limShift)

        ax2 = plt.subplot(1, 2, 2, projection='3d')
        hist, bins = np.histogram(phaseTheta[class1], bins=100, range=(-np.pi, np.pi))
        # print(np.array([np.zeros_like(hist), hist]).shape)
        ax2.plot_surface(
            np.cos(bins[:-1]), np.sin(bins[:-1]), 
            np.array([np.zeros_like(hist), hist]), 
            color='tomato', alpha=0.5, edgecolor="tomato"
        )
        hist, bins = np.histogram(phaseTheta[class2], bins=100, range=(-np.pi, np.pi))
        ax2.plot_surface(
            np.cos(bins[:-1]) + shift, np.sin(bins[:-1]) + shift,
            np.array([np.zeros_like(hist), hist]), 
            color='dodgerblue', alpha=0.5, edgecolor="dodgerblue"
        )
        ax2.set_xlabel(r"$\cos(\theta_I)$")
        ax2.set_ylabel(r"$\sin(\theta_I)$")
        ax2.set_zlabel("Count")
        ax2.set_zlim(0, 1000)

    pbar = tqdm(total=TNum)
    fig, ax = plt.subplots(figsize=(11, 5))
    ani = ma.FuncAnimation(fig, plot_frame, frames=np.arange(0, TNum, 1), interval=50, repeat=False)
    ani.save(f"./mp4/{model}.mp4", dpi=200, writer="ffmpeg")
    
    plt.close()

    pbar.close()

In [ ]:
rangeLambdas = np.concatenate([
    np.arange(0.01, 0.1, 0.02), np.arange(0.1, 1, 0.2)
])
distanceDs = np.concatenate([
    np.arange(0.1, 1, 0.2)
])
rangeGamma = np.concatenate([
    np.arange(0.1, np.pi/2, 0.2)
])

savePath = "./data"

models = [
    ThreeBody(l1, l2, d1, d2, gamma, agentsNum=200, boundaryLength=5,
            tqdm=True, savePath=savePath, overWrite=True)
    for l1, l2, d1, d2, gamma in product(rangeLambdas, rangeLambdas, distanceDs, distanceDs, rangeGamma)
]


In [ ]:
len(models)

In [ ]:
draw_mp4(model)

  0%|          | 0/10002 [00:00<?, ?it/s]